# Modeling mansard roofs

The following workshop aims to model different mansard roofs starting from a .lines file containing the measures of the final roof to achieve. An example of the type of roof which is to be realized is present in the next picture.

## Example:

<img src='img/mansard_roof.jpg'>

## Execution:

To make the final roof the following steps were followed:

    1)It realized the shape of the roof base through .lines file;
    
    2)It made the top of the roof going to resize the previous form;
    
    3)Calculated the centroid of both figures;
    
    4)Translate the forms so as to match the respective centroid;
    
    5)Calculated and ordered the vertex of both figures;
    
    6)Realized the roof pitches;
    
    7)Added top of the roof;

## Support code:

In [ ]:
def make_unique(original_list):
    """Removes duplicates from a list.
       @param original_list: startlist """
    unique_list = []
    [unique_list.append(obj) for obj in original_list if obj not in unique_list]
    return unique_list

def area_of_polygon(x, y):
    """Calculates the signed area of an arbitrary polygon given its verticies.
       @param x: x coordinates of the vertices
       @param y: y coordinates of the vertices """
    area = 0.0
    for i in xrange(-1, len(x) - 1):
        area += x[i] * (y[i + 1] - y[i - 1])
    return area / 2.0

def centroid_of_polygon(points):
    """Calculates the centroid of an arbitrary polygon given its verticies.
       @param points: vertices of an arbitrary polygon """
    area = area_of_polygon(*zip(*points))
    result_x = 0
    result_y = 0
    N = len(points)
    points = cycle(points)
    x1, y1 = next(points)
    for i in range(N):
        x0, y0 = x1, y1
        x1, y1 = next(points)
        cross = (x0 * y1) - (x1 * y0)
        result_x += (x0 + x1) * cross
        result_y += (y0 + y1) * cross
    result_x /= (area * 6.0)
    result_y /= (area * 6.0)
    return (result_x, result_y)

def clockwise_order(points):
    """Order the vertices of a polygon in a clockwise direction from the first vertex in the list.
       @param points: vertices of an arbitrary polygon """
	firstPoint = points[0]
	centroid = [firstPoint[0],firstPoint[1]+0.0000001]
	print centroid
	firstLength = distance(centroid,firstPoint)
	new_points = []

	while len(points)>0:
		cos = 400
		for j in range(len(points)):
			cos2 = find_angle(points[j],centroid)
			
			if cos2 < cos:
				cos = cos2
				new_add = points[j]
		new_points.append(new_add)
		for t in range(len(points)):
			if points[t]==new_add:
				el = t
		
		points.pop(el)
		

	return new_points 

def distance(p1,p2):
    """Calculates the distance between two vertices.
        @param p1: first vertex
        @param p2: second vertex """
	return float(math.sqrt(math.pow((p1[0] - p2[0]), 2)+math.pow((p1[1] - p2[1]), 2)))

def find_angle(p1,p2):
    """Calculates the angle between two vertices through arctangent function with two arguments.
        @param p1: first vertex
        @param p2: second vertex """
	return math.atan2(p1[1]-p2[1],p1[0]-p2[0])

## Main code:

In [ ]:
def builder_roof(fileName):
	j=0
	s=0
	t=0
	first_shape = []
	falde = []
	top_shape = []
	pol= []
	pol2 = []
    """Reading .lines files to achieve the shape and the centroid of the roof"""
	with open("polygon/"+fileName +  ".lines", "rb") as file:
		reader = csv.reader(file, delimiter=",")
		polylineList = []
		reader2 = reader
		row1=next(reader2)
		px = row1[0]
		py = row1[1]
	with open("polygon/"+fileName +  ".lines", "rb") as file:
		reader = csv.reader(file, delimiter=",")
		for row in reader:
			first_shape.append([float(row[0])-float(px),float(row[1])-float(py)])
			first_shape.append([float(row[2])-float(px),float(row[3])-float(py)])
	centroid = centroid_of_polygon(first_shape)
	[x,y] = centroid
	centroid = [x,y]
    """Translational first shape"""
	for f in range(len(first_shape)):
		first_shape[f][0]=first_shape[f][0]-centroid[0]
		first_shape[f][1]=first_shape[f][1]-centroid[1]
    """Modeling firts shape"""
	while j<len(first_shape):
		pol.append(POLYLINE([first_shape[j],first_shape[j+1]]))
		j=j+2
	pol = STRUCT(pol)

	"""Modeling top of the roof"""
	for i in range(len(first_shape)):
		top_shape.append([first_shape[i][0]/2.0,first_shape[i][1]/2.0])

	first_shape = make_unique(first_shape)

	top_shape = make_unique(top_shape)
	top_shape= clockwise_order(top_shape)
	first_shape = clockwise_order(first_shape)
	top_shape = top_shape + [top_shape[0]]
	while t<len(top_shape)-1:
		pol2.append(POLYLINE([top_shape[t],top_shape[t+1]]))
		t=t+1

	piano = SOLIDIFY(STRUCT(pol2))
	piano = T(3)(10)(piano)
	first_shape = first_shape+ [first_shape[0]]
	for p in range(len(first_shape)):
		first_shape[p]=first_shape[p]+[float(0)]
	for k in range(len(top_shape)):
		top_shape[k]=top_shape[k]+[float(10)]
    """Construction flaps"""
	while s<len(make_unique(first_shape)):
		falde.append(MKPOL([[first_shape[s],first_shape[s+1],top_shape[s],top_shape[s+1]],[[1,2,3,4]],None]))
		s = s+1
	falde = STRUCT(falde)
    
    """The HPC model of the generated roof"""
	return STRUCT([falde,piano])

## Results:

The following images show the execution of the main code to build roofs with a different number of layers, concave or convex.

<img src='img/l_roof.jpg'>

<img src='img/roof1.jpg'>

<img src='img/2roof.jpg'>

<img src='img/3roof.jpg'>